# ABOUT:
- given an address "raya samb gede, 299 toko bb kids" we use our fine tuned models to predict if
    1. POI exists or does not exist
    2. street exists or does not exist

## read test set

In [1]:
# read test set > convert to Dataset
import pandas as pd
df = pd.read_csv(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\datasets\test.csv\test.csv")#,index_col = 0)
from datasets import Dataset
test_set_Dataset = Dataset.from_pandas(df[["id","raw_address"]])
del df

In [2]:
test_set_Dataset

Dataset({
    features: ['id', 'raw_address'],
    num_rows: 50000
})

## tokenize test set

In [3]:
model_checkpoint = "w11wo/indonesian-roberta-base-sentiment-classifier"

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, 
                                          use_fast=True)

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["raw_address"], 
                     truncation = True,
                     padding = "max_length",  
                     max_length=50,
                     )
                     

In [6]:
tokenized_test_set = test_set_Dataset.map(preprocess_function,batched=True)

In [7]:
tokenized_test_set

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'raw_address'],
    num_rows: 50000
})

In [8]:
import datasets
import pandas as pd
from IPython.display import display, HTML
def show_elements(dataset, indices=[0,1,2]):
    df = pd.DataFrame(dataset[indices])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_elements(tokenized_test_set)

,attention_mask,id,input_ids,raw_address
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[87, 18, 2847, 16420, 266, 5761, 3727, 902, 26790, 16906, 1850, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",s. par 53 sidanegara 4 cilacap tengah
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,"[511, 328, 16, 2777, 10129, 2164, 539, 18, 32128, 7904, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","angg per, baloi indah kel. lubuk baja"
2,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2,"[279, 1674, 21387, 16, 12172, 2668, 562, 16, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","asma laun, mand imog,"


## Make predictions: POI existence

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\models\POI existence model")

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig
trainer = Trainer(
    model
)

In [13]:
import numpy as np
output = trainer.predict(tokenized_test_set)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, raw_address.
***** Running Prediction *****
  Num examples = 50000
  Batch size = 8


In [26]:
# output logits
output.predictions

array([[ 2.9536135 , -2.629752  ],
       [ 2.9204834 , -2.5633223 ],
       [-1.4248106 ,  0.6736819 ],
       ...,
       [-1.42799   ,  0.6801097 ],
       [-1.4171407 ,  0.61941385],
       [-0.33827168, -0.31427923]], dtype=float32)

In [27]:
# convert logits to probs
from scipy.special import softmax
probabilities = softmax(output.predictions, axis=1)
probabilities

array([[0.9962543 , 0.00374581],
       [0.9958637 , 0.00413631],
       [0.10924342, 0.89075655],
       ...,
       [0.10831206, 0.8916879 ],
       [0.11541805, 0.88458204],
       [0.4940022 , 0.50599784]], dtype=float32)

In [37]:
# convert probs to labels
POI_existence = list(map(np.argmax,probabilities))
print(POI_existence[:10])

[0, 0, 1, 1, 0, 1, 1, 1, 1, 0]


## Make predictions: street existence

In [34]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\models\street existence model")

loading configuration file C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\models\street existence model\config.json
Model config RobertaConfig {
  "_name_or_path": "indonesian-roberta-base-sentiment-classifier",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.8.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file C:\Users\tanch\D

In [35]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig
trainer = Trainer(
    model
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
import numpy as np
output = trainer.predict(tokenized_test_set)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, raw_address.
***** Running Prediction *****
  Num examples = 50000
  Batch size = 8


In [39]:
# output logits
output.predictions

array([[-1.2884191 ,  0.92457765],
       [-1.3210913 ,  0.96678615],
       [-1.0882895 ,  0.7414263 ],
       ...,
       [-1.1866181 ,  0.84084505],
       [-1.2925812 ,  0.9363627 ],
       [-1.1133329 ,  0.78417796]], dtype=float32)

In [40]:
# convert logits to probs
from scipy.special import softmax
probabilities = softmax(output.predictions, axis=1)
probabilities

array([[0.09858945, 0.90141064],
       [0.09213194, 0.9078681 ],
       [0.13827212, 0.8617279 ],
       ...,
       [0.11634947, 0.8836505 ],
       [0.09718128, 0.90281874],
       [0.13039045, 0.8696095 ]], dtype=float32)

In [42]:
# convert probs to labels
street_existence = list(map(np.argmax,probabilities))
print(street_existence[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## export

In [53]:
import pickle
with open(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\datasets\street_existence", "wb") as f:
    pickle.dump(street_existence,f)

In [54]:
with open(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\datasets\POI_existence", "wb") as f:
    pickle.dump(POI_existence,f)

In [52]:
street_existence

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
